In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [3]:
gc.collect()

items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
#sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train[sales_train['date_block_num'] > 26]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [5]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_keep = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()
#ids_keep = sums[(sums['item_total_sales'] > 0)]['item_id'].unique()

In [6]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
#train_item_ids = ids_keep
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

#all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_item_ids = test_item_ids

#all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))
all_shop_ids = test_shop_ids



In [7]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [8]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [9]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [10]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [11]:
training['item_cnt_block_mean'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.mean)
training['item_cnt_block_min'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.min)
training['item_cnt_block_max'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.max)
training['item_cnt_block_std'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.std)
training['item_cnt_block_med'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.median)

In [12]:
training['shop_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.mean)
training['shop_cnt_block_min'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.min)
training['shop_cnt_block_max'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.max)
training['shop_cnt_block_std'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.std)
training['shop_cnt_block_med'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.median)

In [13]:
training['category_cnt_block_mean'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.mean)
training['category_cnt_block_min'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.min)
training['category_cnt_block_max'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.max)
training['category_cnt_block_std'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.std)
training['category_cnt_block_med'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.median)

In [14]:
training['shop_category_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.mean)
training['shop_category_cnt_block_min'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.min)
training['shop_category_cnt_block_max'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.max)
training['shop_category_cnt_block_std'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.std)
training['shop_category_cnt_block_med'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.median)

In [15]:
training['shop_item_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.mean)
training['shop_item_cnt_block_min'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.min)
training['shop_item_cnt_block_max'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.max)
training['shop_item_cnt_block_std'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.std)
training['shop_item_cnt_block_med'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.median)

In [16]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id"]



y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [17]:
training.fillna(0,inplace=True)

In [18]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,category_cnt_block_mean,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med,shop_item_cnt_block_mean,shop_item_cnt_block_min,shop_item_cnt_block_max,shop_item_cnt_block_std,shop_item_cnt_block_med,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
774766,11631,16,33,0,37,10,2015,3.0,963.0,4061.0,37.0,13.289608,0.0,4078.0,75.376194,4.0,1553.785714,330.0,6247.0,1336.398262,1161.0,3228.865882,0.0,7680.0,2663.646850,2290.0,73.363525,0.0,2524.0,135.813744,30.0,0.255649,0,20,1.089856,0,0.021008,0.182890,0.178702
185571,2951,14,28,0,19,5,2015,2.0,933.0,3392.0,43.0,11.949804,0.0,3768.0,89.593827,2.0,1592.166667,0.0,6327.0,1311.452767,1211.5,3428.631373,0.0,9208.0,3303.055672,1635.0,75.539594,-1.0,2005.0,147.855428,26.0,0.213301,0,20,1.018289,0,0.052174,0.158941,0.520958
1128291,16093,46,30,0,64,7,2015,0.0,1642.0,1076.0,45.0,10.808824,-1.0,3347.0,53.842045,3.0,1427.642857,0.0,5987.0,1114.915109,1108.0,3335.517843,0.0,9283.0,3239.632607,1890.0,75.583501,-1.0,1475.0,141.362130,32.0,0.227605,0,20,0.949685,0,0.000000,0.242465,0.159940
253732,3777,3,30,0,55,7,2015,0.0,535.0,6474.0,48.0,10.808824,-1.0,3347.0,53.842045,3.0,1427.642857,0.0,5987.0,1114.915109,1108.0,3335.517843,0.0,9283.0,3239.632607,1890.0,75.583501,-1.0,1475.0,141.362130,32.0,0.227605,0,20,0.949685,0,0.041494,0.119328,0.194614
163930,2722,38,31,0,31,8,2015,0.0,1781.0,724.0,0.0,11.840196,-1.0,3551.0,57.770225,3.0,1551.500000,0.0,5714.0,1090.984155,1271.0,3253.890392,0.0,8513.0,2995.889840,1788.0,75.921218,0.0,1189.0,128.648912,32.0,0.260037,0,20,1.031644,0,0.000000,0.226773,0.046200
1031555,15067,45,27,0,29,4,2015,0.0,862.0,291.0,1.0,12.535490,-1.0,7300.0,124.515785,2.0,1711.166667,0.0,7341.0,1447.095173,1309.5,4010.790784,0.0,14751.0,4102.264551,2285.0,88.694935,-1.0,2506.0,195.630747,26.0,0.215145,0,20,1.079221,0,0.978448,0.118249,0.254048
183101,2921,49,29,1,21,6,2015,44.0,789.0,208.0,6.0,10.833333,-1.0,3473.0,58.426138,2.0,1430.904762,0.0,6160.0,1194.835848,1058.0,3318.272157,0.0,9304.0,3228.111861,1919.0,74.266709,0.0,1529.0,150.766699,29.0,0.221471,0,20,1.008618,0,0.762500,0.125294,0.329041
491114,7094,31,28,0,31,5,2015,0.0,6327.0,1138.0,0.0,11.949804,0.0,3768.0,89.593827,2.0,1592.166667,0.0,6327.0,1311.452767,1211.5,3428.631373,0.0,9208.0,3303.055672,1635.0,75.539594,-1.0,2005.0,147.855428,26.0,0.213301,0,20,1.018289,0,0.000000,0.889627,0.043165
1136578,16129,56,29,0,64,6,2015,2.0,1566.0,897.0,10.0,10.833333,-1.0,3473.0,58.426138,2.0,1430.904762,0.0,6160.0,1194.835848,1058.0,3318.272157,0.0,9304.0,3228.111861,1919.0,74.266709,0.0,1529.0,150.766699,29.0,0.221471,0,20,1.008618,0,0.081197,0.213913,0.160936
332893,4865,19,28,0,31,5,2015,2.0,1459.0,1138.0,0.0,11.949804,0.0,3768.0,89.593827,2.0,1592.166667,0.0,6327.0,1311.452767,1211.5,3428.631373,0.0,9208.0,3303.055672,1635.0,75.539594,-1.0,2005.0,147.855428,26.0,0.213301,0,20,1.018289,0,0.231405,0.237185,0.043224


In [19]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max', 'shop_category_cnt_block_std',
       'shop_category_cnt_block_med', 'shop_item_cnt_block_mean',
       'shop_item_cnt_block_min', 'shop_item_cnt_block_max',
       'shop_item_cnt_block_std', 'shop_item_cnt_block_med',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_ca

In [32]:
all_features = [
    
  'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max', 'shop_category_cnt_block_std',
       'shop_category_cnt_block_med', 'shop_item_cnt_block_mean',
       'shop_item_cnt_block_min', 'shop_item_cnt_block_max',
       'shop_item_cnt_block_std', 'shop_item_cnt_block_med',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'
]



features =  [
    
    'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'
    
]

In [21]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 


training[all_features] = MinMaxScaler().fit_transform(training[all_features])



/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [22]:
training[all_features] = training[all_features].apply(pd.to_numeric, downcast='float')

In [33]:
training[features].sample(10)

,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,category_cnt_block_mean,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
828723,0.004109,0.073832,0.485526,0.025347,0.696016,0.0,1.000000,1.000000,0.0,0.971368,0.0,1.000000,0.705080,1.000000,1.000000,0.0,1.000000,1.000000,0.992366,1.000000,0.0,0.987543,0.021481,0.130024,0.011119
579632,0.003150,0.000000,0.577113,0.000396,0.415745,0.0,0.051606,0.055582,0.5,0.424341,0.0,0.000000,0.000000,0.846918,0.359440,0.0,0.183935,0.325607,0.233588,0.426696,1.0,0.076125,0.021216,0.000000,0.011242
678408,0.000274,0.176951,0.019456,0.000396,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.0,0.167793,0.047382,0.198807,0.428521,0.0,0.284668,0.474181,0.389313,0.411539,0.0,0.274048,0.001751,0.259450,0.002846
540729,0.000137,0.185806,0.026371,0.000792,0.696016,0.0,1.000000,1.000000,0.0,0.971368,0.0,1.000000,0.705080,1.000000,1.000000,0.0,1.000000,1.000000,0.992366,1.000000,0.0,0.987543,0.001469,0.236781,0.001326
1412204,0.001096,0.137175,0.095655,0.005545,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.0,0.167793,0.047382,0.198807,0.428521,0.0,0.284668,0.474181,0.389313,0.411539,0.0,0.274048,0.008288,0.218155,0.007754
806733,0.000411,0.165373,0.015660,0.000396,0.415745,0.0,0.051606,0.055582,0.5,0.424341,0.0,0.000000,0.000000,0.846918,0.359440,0.0,0.183935,0.325607,0.233588,0.426696,1.0,0.076125,0.001878,0.176729,0.018614
1064048,0.000959,0.136494,0.145685,0.010297,1.000000,1.0,0.184923,0.304698,1.0,0.432172,1.0,0.327597,0.485906,0.409543,0.338262,0.0,0.074961,0.123088,1.000000,0.311903,1.0,1.000000,0.023909,0.167711,0.014646
1029529,0.004520,0.131590,0.053285,0.008713,0.415745,0.0,0.051606,0.055582,0.5,0.424341,0.0,0.000000,0.000000,0.846918,0.359440,0.0,0.183935,0.325607,0.233588,0.426696,1.0,0.076125,0.018928,0.132848,0.004083
1470352,0.000137,0.138809,0.254627,0.029703,0.009880,0.0,0.031875,0.064863,0.0,0.011175,0.0,0.274124,0.205621,0.000000,0.413926,0.0,0.287415,0.467158,0.433588,0.352439,1.0,0.311419,0.000000,0.167501,0.010263
756168,0.000274,0.117014,1.000000,0.092673,0.696016,0.0,1.000000,1.000000,0.0,0.971368,0.0,1.000000,0.705080,1.000000,1.000000,0.0,1.000000,1.000000,0.992366,1.000000,0.0,0.987543,0.002458,0.140748,0.051816


In [34]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3).fit(training[features])
print(pca.explained_variance_ratio_)

[0.38815073 0.25422475 0.15302156]


In [35]:
training_pca = pca.transform(training[features])

In [36]:
training['pca1'] = np.array(training_pca).T[0]
training['pca2'] = np.array(training_pca).T[1]
training['pca3'] = np.array(training_pca).T[2]

In [52]:
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['shop_item_cnt_block']
x_train = x_train[['date_block_num', 'pca1', 'pca2', 'pca3']]




x_val = training[training['date_block_num'] == 33]
y_val = x_val['item_cnt_block']
x_val = x_val[['date_block_num', 'pca1', 'pca2', 'pca3']]

In [53]:
len(x_train)

1285200

In [56]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor


lr_model =  LinearRegression()
lr_model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  0.3094134923280002


In [55]:
lr_model.coef_

array([0.01222557, 0.00579536, 0.01137676, 0.01917079])

In [ ]:
test.head()

In [ ]:
training[(training['shop_id'] == 5) & ((training['item_id'] == 5037))][['date_block_num', 'pca1', 'pca2', 'pca3']]

In [44]:
training_test = training[(training['shop_id'].isin(test['shop_id'].unique()))\
                         & (training['item_id'].isin(test['item_id'].unique()))]

In [45]:
a = list(range(0, 235620, 21420))
b = list(range(21420, 257040, 21420))
intervals = list(zip(a,b))[:-1]
intervals

[(0, 21420),
 (21420, 42840),
 (42840, 64260),
 (64260, 85680),
 (85680, 107100),
 (107100, 128520),
 (128520, 149940),
 (149940, 171360),
 (171360, 192780),
 (192780, 214200)]

In [57]:
import multiprocessing as mp

import importlib
import build_test_lr
importlib.reload(build_test_lr)


from build_test_lr import build_test_lr_f

features = ['pca1', 'pca2', 'pca3']

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_test_lr_f,args=[interval, test, training_test, features]) for interval in intervals]
    pool.close()
    pool.join()

(0, 21420)
(21420, 42840)
(42840, 64260)
(64260, 85680)
(85680, 107100)
(107100, 128520)
(128520, 149940)
(149940, 171360)
(171360, 192780)
(192780, 214200)


In [58]:
test_data = []

for interval in intervals:
    for re in res:
        if interval in re.get():
            for sample in re.get()[interval]:
                test_data.append(sample)

In [59]:
for i,component in enumerate(pca.explained_variance_ratio_):
    name = 'pca%d' % (i)
    test[name] = np.array(test_data).T[i]
    
test['date_block_num'] =  34

In [60]:
preds = lr_model.predict(test[['date_block_num','pca1', 'pca2', 'pca3']])
preds.clip(0,20,out=preds)

array([0.30386724, 0.30390692, 0.30386722, ..., 0.30393215, 0.30396544,
       0.30389923])

In [61]:

print(np.mean(preds))
print(np.max(preds))

0.3038352155356227
0.3039999506808465


In [62]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
bestpreds = pd.read_csv('submissionbest.csv')['item_cnt_month']
print(np.mean(bestpreds))
print(np.max(bestpreds))

In [ ]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)